# Parse raw data into Dataframes

**[Johnnatan Messias](https://johnnatan-messias.github.io), March 2025**


In [ ]:
# Standard Library Imports
import gzip
import pickle

# Third-Party Libraries
from tqdm.notebook import tqdm
import pandas as pd
import polars as pl

# Polars Configuration
pl.Config.set_float_precision(18)

In [2]:
import sys
import os
code_dir = os.path.realpath(os.path.join(os.getcwd(), "..", "src"))

sys.path.append(code_dir)

In [3]:
from utils import Utils
contract_settings = Utils.load_contract_settings(chain="mainnet")
print(contract_settings.keys())

dict_keys(['ens', 'ens_register', 'dydx', 'dydx-merkle-distributor', 'compound', 'uniswap', 'sushi', '1inch', 'tornadocash', 'worldcoin', 'lido', 'lido-token', 'arkham-token', 'arkham-airdrop'])


In [4]:
path_data = os.path.realpath(os.path.join(os.getcwd(), "..", "data"))
path_plots = os.path.realpath(os.path.join(os.getcwd(), "..", "plots"))
os.makedirs(path_data, exist_ok=True)
os.makedirs(path_plots, exist_ok=True)

# Data parsing


In [5]:
contract_names = sorted(['_'.join(filename.split('_')[1:])[:-7] for filename in os.listdir(
    path_data) if filename.startswith("events_") and filename.endswith(".pkl.gz")])
contract_names

['1inch',
 'arkham-airdrop',
 'arkham-token',
 'compound',
 'dydx',
 'dydx-merkle-distributor',
 'ens',
 'ens_register',
 'lido',
 'lido-token',
 'sushi',
 'tornadocash',
 'uniswap',
 'worldcoin']

In [6]:
def load_dataset(contract_name):
    file_path = os.path.realpath(os.path.join(
        path_data, 'events_{}.pkl.gz'.format(contract_name)))
    with gzip.open(file_path, 'rb') as f:
        events = pickle.load(f)
    return events


def load_transfer(protocol_name, amount_column="amount", sender="from", receiver="to"):
    data = load_dataset(protocol_name)
    transfer_pl = Utils.transfer_to_dataframe(
        data['Transfer'], amount_colum=amount_column, sender=sender, receiver=receiver)
    file_path = os.path.realpath(os.path.join(path_data, "blocks.parquet.gz"))
    blocks_pl = (pl.scan_parquet(file_path)
                 .select(['number', 'timestamp'])).collect()
    transfer_pl = (transfer_pl
                   .join(blocks_pl, left_on='blockNumber', right_on='number')
                   .sort(['blockNumber', 'transactionIndex', 'logIndex'])
                   )
    return transfer_pl


def load_claim(protocol_name, amount_column="amount", event_claim="Claim", account_column="account"):
    data = load_dataset(protocol_name)
    claim_pl = Utils.claim_to_dataframe(
        data[event_claim], amount_colum=amount_column, account_column=account_column)
    file_path = os.path.realpath(os.path.join(path_data, "blocks.parquet.gz"))
    blocks_pl = (pl.scan_parquet(file_path)
                 .select(['number', 'timestamp'])).collect()
    claim_pl = (claim_pl
                .join(blocks_pl, left_on='blockNumber', right_on='number')
                .sort(['blockNumber', 'transactionIndex', 'logIndex'])
                )
    return claim_pl

### Parsing blocks


In [7]:
file_path = os.path.realpath(os.path.join(path_data, "blocks.parquet.gz"))
blocks_pl = (pl.scan_parquet(file_path)
             .select(['number', 'timestamp'])).collect()

## 1inch


In [8]:
protocol_name = "1inch"
one_inch_transfer_pl = load_transfer(protocol_name, amount_column="value")
one_inch_transfer_pl = one_inch_transfer_pl.with_columns(pl.col('amount').truediv(
    10**contract_settings[protocol_name]['decimals']).alias('amount'))
print('There are {} Transfer events'.format(len(one_inch_transfer_pl)))
one_inch_transfer_pl.head()

Loading Transfer events: 100%|██████████| 1870426/1870426 [00:02<00:00, 708417.64it/s]


There are 1782998 Transfer events


blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,from,to,amount,timestamp
i64,str,str,str,i64,i64,str,str,str,f64,datetime[ns]
11511393,"""9825b9f083ba43112afd66dbcfbbf7…","""e6fcb64146f00803f902a8d6b42a6d…","""0x111111111117dc0aa78b770fa6a7…",2,3,"""Transfer""","""0x0000000000000000000000000000…","""0x5e89f8d81c74e311458277ea1be3…",1.500000000000000000e9,2020-12-23 18:13:31
11519469,"""27a7d7804d4d3062624df2497ca7ca…","""cdfc2690772d031aef1716916672f0…","""0x111111111117dc0aa78b770fa6a7…",137,330,"""Transfer""","""0x5e89f8d81c74e311458277ea1be3…","""0xe295ad71242373c37c5fda7b57f2…",9.000000000000001490e7,2020-12-25 00:05:12
11519481,"""a93bd3f64439263ac24e790eb11555…","""03341cc7ac7f0882bfa830ce7b43fa…","""0x111111111117dc0aa78b770fa6a7…",166,315,"""Transfer""","""0xe295ad71242373c37c5fda7b57f2…","""0x68a17b587caf4f9329f0e372e3a7…",3.101095064658825868e4,2020-12-25 00:08:54
11519484,"""5a5320dc7763c45a2eeed15e17da31…","""9bfc017f196151209b4e5bbbf2c69e…","""0x111111111117dc0aa78b770fa6a7…",93,213,"""Transfer""","""0x68a17b587caf4f9329f0e372e3a7…","""0xa09d1afae21bda81c06fd6de9bbd…",1.000000000000000114e3,2020-12-25 00:09:35
11519486,"""6183751977a20377add8cf002a5888…","""1fb78c77aa46ba205bbcb166231e53…","""0x111111111117dc0aa78b770fa6a7…",220,281,"""Transfer""","""0xe295ad71242373c37c5fda7b57f2…","""0xaa9e20bab58d013220d632874e9f…",1.824738794374177814e3,2020-12-25 00:10:05


In [9]:
one_inch_claiming_address = "0xe295ad71242373c37c5fda7b57f26f9ea1088afe"
one_inch_claim_pl = one_inch_transfer_pl.filter(
    pl.col("from").eq(one_inch_claiming_address))
print('There are {} Claim events'.format(len(one_inch_claim_pl)))
one_inch_claim_pl.head()

There are 43806 Claim events


blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,from,to,amount,timestamp
i64,str,str,str,i64,i64,str,str,str,f64,datetime[ns]
11519481,"""a93bd3f64439263ac24e790eb11555…","""03341cc7ac7f0882bfa830ce7b43fa…","""0x111111111117dc0aa78b770fa6a7…",166,315,"""Transfer""","""0xe295ad71242373c37c5fda7b57f2…","""0x68a17b587caf4f9329f0e372e3a7…",3.101095064658825868e4,2020-12-25 00:08:54
11519486,"""6183751977a20377add8cf002a5888…","""1fb78c77aa46ba205bbcb166231e53…","""0x111111111117dc0aa78b770fa6a7…",220,281,"""Transfer""","""0xe295ad71242373c37c5fda7b57f2…","""0xaa9e20bab58d013220d632874e9f…",1.824738794374177814e3,2020-12-25 00:10:05
11519493,"""05b2208f9399abd73b6760653f45c8…","""308d8b54560fb193ab2097d5a6ccc3…","""0x111111111117dc0aa78b770fa6a7…",81,157,"""Transfer""","""0xe295ad71242373c37c5fda7b57f2…","""0xaaa3bfb53d5d5116faedc5dd4575…",2.570116004920753767e3,2020-12-25 00:11:52
11519496,"""a74fab4006d9b4e815a2e9407111df…","""bb45238736aab3481f08916eef126a…","""0x111111111117dc0aa78b770fa6a7…",26,38,"""Transfer""","""0xe295ad71242373c37c5fda7b57f2…","""0x93dd644468b5351bc419ddf3a5bb…",6.245036586325297776e2,2020-12-25 00:12:29
11519499,"""3f3d2b343db6adf818b030db0ff6e3…","""91b517978063121fdc81898fd574b2…","""0x111111111117dc0aa78b770fa6a7…",0,0,"""Transfer""","""0xe295ad71242373c37c5fda7b57f2…","""0xa793c481125d93abad899e4fdb75…",1.387886718790358600e3,2020-12-25 00:12:53


In [10]:
file_path = os.path.realpath(os.path.join(path_data, "1inch_transfer.parquet"))
one_inch_transfer_pl.write_parquet(file_path, compression="gzip")

file_path = os.path.realpath(os.path.join(path_data, "1inch_claim.parquet"))
one_inch_claim_pl.write_parquet(file_path, compression="gzip")

## Arkham


In [11]:
protocol_name = 'arkham-token'
arkham_transfer_pl = load_transfer(protocol_name, amount_column="value")
arkham_transfer_pl = arkham_transfer_pl.with_columns(pl.col('amount').truediv(
    10**contract_settings[protocol_name]['decimals']).alias('amount'))
print('There are {} Transfer events'.format(len(arkham_transfer_pl)))
arkham_transfer_pl.head()

Loading Transfer events: 100%|██████████| 341500/341500 [00:00<00:00, 766009.38it/s]


There are 283775 Transfer events


blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,from,to,amount,timestamp
i64,str,str,str,i64,i64,str,str,str,f64,datetime[ns]
17628655,"""996484eb3697465c24b8bce79cf917…","""a9e0d87c7e297b17f181d6bb5f181d…","""0x6e2a43be0b1d33b726f0ca3b8de6…",17,90,"""Transfer""","""0x0000000000000000000000000000…","""0x818f0dff37015c77cfe5686c35ed…",1.000000000000000119e9,2023-07-05 15:44:59
17634265,"""884edef75b78b122f21e800b70406e…","""f581920a4a381c1a2fa16fd1f1d1a4…","""0x6e2a43be0b1d33b726f0ca3b8de6…",56,171,"""Transfer""","""0x818f0dff37015c77cfe5686c35ed…","""0xcd5e3ccd9594a70dc6669a9b7715…",8.990485369333630800e7,2023-07-06 10:38:23
17634266,"""9fe84eceb14a15f95f84ef63a84947…","""1d7f11357679c6c8dbbfdb3c2c6b12…","""0x6e2a43be0b1d33b726f0ca3b8de6…",39,74,"""Transfer""","""0x818f0dff37015c77cfe5686c35ed…","""0x9aed8c8a5313f1c57ab4ba73bef6…",8.990485369333630800e7,2023-07-06 10:38:35
17634267,"""fecca57de38519e1ccbde506f7e3ce…","""01327d6dc0589bb76b5f43c7bea54c…","""0x6e2a43be0b1d33b726f0ca3b8de6…",58,159,"""Transfer""","""0x818f0dff37015c77cfe5686c35ed…","""0x2c09dbfa73fed7c4820f96136d72…",8.670823667312880978e6,2023-07-06 10:38:47
17634268,"""476f72d2ba6ab257dae0176fd51474…","""21e3bac88b45d3e372a22a7aab8dad…","""0x6e2a43be0b1d33b726f0ca3b8de6…",67,127,"""Transfer""","""0x818f0dff37015c77cfe5686c35ed…","""0xb3b5e085b45f8c423d3231b4a718…",2.155219131593040191e6,2023-07-06 10:38:59


In [12]:
protocol_name = 'arkham-airdrop'
arkham = load_dataset(protocol_name)
arkham_claim_pl = Utils.claim_to_dataframe(arkham['Claimed'])
arkham_claim_pl = (arkham_claim_pl
                   .join(blocks_pl, left_on='blockNumber', right_on='number')
                   .sort(['blockNumber', 'transactionIndex', 'logIndex'])
                   .with_columns(pl.col('amount').truediv(
                       10**contract_settings[protocol_name]['decimals']).alias('amount')))
print('There are {} Claimed events'.format(len(arkham_claim_pl)))
arkham_claim_pl.head()

Loading Claim events: 100%|██████████| 64732/64732 [00:00<00:00, 851320.50it/s]

There are 64724 Claimed events


blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,account,amount,timestamp
i64,str,str,str,i64,i64,str,str,f64,datetime[ns]
17718694,"""0470a8b26c76a982004f888bc32146…","""c40220b81ee0ae287cc5ead6ce4bb0…","""0x08c7676680f187a31241e83e6d44…",24,28,"""Claimed""","""0x67c1bb9d5408655bacd89773f553…",1.974294553296443553e2,2023-07-18 07:41:35
17718695,"""d9ce2b189ee75199f235f6d49b51c7…","""c19bfaa69b21e8f0a3fd9a61c36226…","""0x08c7676680f187a31241e83e6d44…",29,121,"""Claimed""","""0x98527d130586a646ebbd29c94244…",1.776865097966799112e3,2023-07-18 07:41:47
17718695,"""be996117f6db5ef0909184fcb2d904…","""c19bfaa69b21e8f0a3fd9a61c36226…","""0x08c7676680f187a31241e83e6d44…",30,123,"""Claimed""","""0x18b5f3724e02762db2698949bfb5…",4.146018561922531262e3,2023-07-18 07:41:47
17718695,"""a680b0b4902b86ae44dac05edcc6b2…","""c19bfaa69b21e8f0a3fd9a61c36226…","""0x08c7676680f187a31241e83e6d44…",31,125,"""Claimed""","""0x431ccb866a7aa303097e87297340…",1.974294553296443553e2,2023-07-18 07:41:47
17718696,"""f46b1ae575444ea02f54ae79343ee8…","""1fd7bd6d7cb413170c4336b64b7d08…","""0x08c7676680f187a31241e83e6d44…",38,105,"""Claimed""","""0x14a130d2fcc0c2a453b0e7d3be75…",2.171724008626088107e3,2023-07-18 07:41:59


In [13]:
file_path = os.path.realpath(os.path.join(
    path_data, "arkham_transfer.parquet"))
arkham_transfer_pl.write_parquet(file_path, compression="gzip")

file_path = os.path.realpath(os.path.join(path_data, "arkham_claim.parquet"))
arkham_claim_pl.write_parquet(file_path, compression="gzip")

## Uniswap


In [14]:
protocol_name = 'uniswap'
uniswap_transfer_pl = load_transfer(protocol_name, amount_column="amount")
uniswap_transfer_pl = uniswap_transfer_pl.with_columns(pl.col('amount').truediv(
    10**contract_settings[protocol_name]['decimals']).alias('amount'))
print('There are {} Transfer events'.format(len(uniswap_transfer_pl)))
uniswap_transfer_pl.head()

Loading Transfer events: 100%|██████████| 5308596/5308596 [00:06<00:00, 813282.92it/s]


There are 4872925 Transfer events


blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,from,to,amount,timestamp
i64,str,str,str,i64,i64,str,str,str,f64,datetime[ns]
10861674,"""4b37d2f343608457ca3322accdab28…","""32e4dd857b5b7e756551a00271e44b…","""0x1f9840a85d5af5bf1d1762f925bd…",9,23,"""Transfer""","""0x0000000000000000000000000000…","""0x41653c7d61609d856f29355e404f…",1.000000000000000119e9,2020-09-14 18:11:26
10861766,"""0169d8b766b66447e7f869ea469805…","""51a1a8789536990bcca505f514e03d…","""0x1f9840a85d5af5bf1d1762f925bd…",65,202,"""Transfer""","""0x41653c7d61609d856f29355e404f…","""0xe5737257d9406019768167c26f5c…",2.000000000000000000e0,2020-09-14 18:31:56
10861777,"""f8edbd74ef0ab3da0b0b2303aab50a…","""e449f5bea40fb94c014fc78977a389…","""0x1f9840a85d5af5bf1d1762f925bd…",76,158,"""Transfer""","""0x41653c7d61609d856f29355e404f…","""0xe5737257d9406019768167c26f5c…",8.000000000000000000e0,2020-09-14 18:35:18
10862390,"""62f4b047a954be261b9e76c1195166…","""b4fcd45357272fe8e2a7c5564505cc…","""0x1f9840a85d5af5bf1d1762f925bd…",64,218,"""Transfer""","""0xe5737257d9406019768167c26f5c…","""0x1c8bca22fa5f57c3e289ed469bca…",2.000000000000000000e0,2020-09-14 20:57:34
10862404,"""2ca0f926e58a9a3f195702726b5250…","""e2115ea5048ff28af165c3637c4793…","""0x1f9840a85d5af5bf1d1762f925bd…",55,75,"""Transfer""","""0xe5737257d9406019768167c26f5c…","""0x557847aa724ea004041522ee550e…",2.000000000000000000e0,2020-09-14 21:00:22


In [15]:
uniswap_claiming_address = "0x090d4613473dee047c3f2706764f49e0821d256e"
uniswap_claim_pl = uniswap_transfer_pl.filter(
    pl.col("from").eq(uniswap_claiming_address))
print('There are {} Claim events'.format(len(uniswap_claim_pl)))
uniswap_claim_pl.head()

There are 221087 Claim events


blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,from,to,amount,timestamp
i64,str,str,str,i64,i64,str,str,str,f64,datetime[ns]
10875578,"""7652efe5d6fd43c075eb186efc77dc…","""1bfa4a2462a331425a8bf557935e0a…","""0x1f9840a85d5af5bf1d1762f925bd…",56,93,"""Transfer""","""0x090d4613473dee047c3f2706764f…","""0x8688a84fcfd84d8f78020d0fc0b3…",1.943093941000000086e3,2020-09-16 21:32:52
10875578,"""593d00f42a3a1056a3ecebc156d6e9…","""1bfa4a2462a331425a8bf557935e0a…","""0x1f9840a85d5af5bf1d1762f925bd…",76,137,"""Transfer""","""0x090d4613473dee047c3f2706764f…","""0xf94e5cdf41247e268d4847c30a0d…",1.400099543000000040e3,2020-09-16 21:32:52
10875631,"""f24757d032a2dde4a6bf4b87dedb08…","""718192737d714b36649df7c92a3b4a…","""0x1f9840a85d5af5bf1d1762f925bd…",90,201,"""Transfer""","""0x090d4613473dee047c3f2706764f…","""0xe79057088a15ac8c9c8bec3b90bd…",4.045821633000000020e3,2020-09-16 21:42:41
10876266,"""f4c2f4503b527e2f1391e162df9998…","""393bed90f37498d139453f9ff3f8d6…","""0x1f9840a85d5af5bf1d1762f925bd…",148,297,"""Transfer""","""0x090d4613473dee047c3f2706764f…","""0x26921a182cf9d6f33730d7f37e1a…",4.114654770000000212e2,2020-09-17 00:05:08
10876273,"""e11241f3d4d4f1192486f77275d431…","""a26e55e4c1d0b7c0009a5262771373…","""0x1f9840a85d5af5bf1d1762f925bd…",121,262,"""Transfer""","""0x090d4613473dee047c3f2706764f…","""0x580eb9591eb975a78ac3de967c7c…",4.000000000000000568e2,2020-09-17 00:06:53


In [16]:
file_path = os.path.realpath(os.path.join(
    path_data, "uniswap_transfer.parquet"))
uniswap_transfer_pl.write_parquet(file_path, compression="gzip")

file_path = os.path.realpath(os.path.join(path_data, "uniswap_claim.parquet"))
uniswap_claim_pl.write_parquet(file_path, compression="gzip")

## ENS


In [17]:
protocol_name = 'ens'
ens_transfer_pl = load_transfer(protocol_name, amount_column="value")
ens_transfer_pl = ens_transfer_pl.with_columns(pl.col('amount').truediv(
    10**contract_settings[protocol_name]['decimals']).alias('amount'))
print('There are {} Transfer events'.format(len(ens_transfer_pl)))
ens_transfer_pl.head()

Loading Transfer events: 100%|██████████| 1148492/1148492 [00:01<00:00, 782009.00it/s]


There are 998619 Transfer events


blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,from,to,amount,timestamp
i64,str,str,str,i64,i64,str,str,str,f64,datetime[ns]
13533418,"""dfc76788b13ab1c033c7cd55fdb7a4…","""bacaa01de0be700ce6d460eb87ee46…","""0xc18360217d8f7ab5e7c516566761…",208,383,"""Transfer""","""0x0000000000000000000000000000…","""0x0904dac3347ea47d208f3fd67402…",7.500000000000000000e7,2021-11-01 20:55:17
13533418,"""dfc76788b13ab1c033c7cd55fdb7a4…","""bacaa01de0be700ce6d460eb87ee46…","""0xc18360217d8f7ab5e7c516566761…",208,384,"""Transfer""","""0x0000000000000000000000000000…","""0xc18360217d8f7ab5e7c516566761…",2.500000000000000373e7,2021-11-01 20:55:17
13533538,"""9492a233a0ec33a5fddb0b63adea2b…","""4f1b4a65be3cfa5ec5bcff167329ca…","""0xc18360217d8f7ab5e7c516566761…",369,612,"""Transfer""","""0x0904dac3347ea47d208f3fd67402…","""0xd7a029db2585553978190db5e85e…",4.500000000000000745e7,2021-11-01 21:19:47
13533540,"""06bb6d2a3353120b05402053950404…","""b2819888adf134142b957fbf2e145e…","""0xc18360217d8f7ab5e7c516566761…",176,265,"""Transfer""","""0x0904dac3347ea47d208f3fd67402…","""0xfe89cc7abb2c4183683ab71653c4…",4.999999999999999069e6,2021-11-01 21:20:18
13534558,"""1938a3b85d114cb0113fb9c450e7b4…","""3cb0e3955be9ad7148a55ee2ad7900…","""0xc18360217d8f7ab5e7c516566761…",15,15,"""Transfer""","""0x0904dac3347ea47d208f3fd67402…","""0x40790db8c3e62f99e24e3fe9c5e9…",1.287300100000000093e5,2021-11-02 00:59:16


In [18]:
protocol_name = 'ens'
ens_claim_pl = load_claim(protocol_name, amount_column="amount",
                          event_claim="Claim", account_column="claimant")
ens_claim_pl = ens_claim_pl.with_columns(pl.col('amount').truediv(
    10**contract_settings[protocol_name]['decimals']).alias('amount'))
print('There are {} Claim events'.format(len(ens_claim_pl)))
ens_claim_pl.head()

Loading Claim events: 100%|██████████| 102830/102830 [00:00<00:00, 783669.38it/s]


There are 102824 Claim events


blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,account,amount,timestamp
i64,str,str,str,i64,i64,str,str,f64,datetime[ns]
13578843,"""9169befc5bb7af4437bcd31569ffcc…","""36c877cbdaed6ba17fd4e0be5561cd…","""0xc18360217d8f7ab5e7c516566761…",69,102,"""Claim""","""0xa753e050948ff5bfb4b695f870b3…",4.306059230120038137e2,2021-11-09 00:01:26
13578844,"""57458e828809dc0efa892a507310cc…","""10365276cb5ea39b91fe2c9acb12e5…","""0xc18360217d8f7ab5e7c516566761…",91,155,"""Claim""","""0x0904dac3347ea47d208f3fd67402…",1.143544204434688481e3,2021-11-09 00:01:43
13578846,"""5a067ac5c5d6aeb5acae38ff66d999…","""651b71e0647a11f3702054ec4b3ed1…","""0xc18360217d8f7ab5e7c516566761…",44,52,"""Claim""","""0x12e245bf546d91bc25fc18c9252c…",1.301280017237164088e2,2021-11-09 00:01:56
13578846,"""a8a0b3030860f4970c6ac4b0e5e3db…","""651b71e0647a11f3702054ec4b3ed1…","""0xc18360217d8f7ab5e7c516566761…",134,187,"""Claim""","""0x3839acf1ee7699d1f46b1be840d8…",7.354902757348929754e2,2021-11-09 00:01:56
13578846,"""6a22895d6e264e72c360251f2145fb…","""651b71e0647a11f3702054ec4b3ed1…","""0xc18360217d8f7ab5e7c516566761…",239,345,"""Claim""","""0x3d597789ea16054a084ac84ce87f…",4.379828076877110448e2,2021-11-09 00:01:56


In [19]:
file_path = os.path.realpath(os.path.join(
    path_data, "ens_transfer.parquet"))
ens_transfer_pl.write_parquet(file_path, compression="gzip")

file_path = os.path.realpath(os.path.join(path_data, "ens_claim.parquet"))
ens_claim_pl.write_parquet(file_path, compression="gzip")

## dYdX


In [20]:
protocol_name = 'dydx'
dydx_transfer_pl = load_transfer(protocol_name, amount_column="value")
dydx_transfer_pl = dydx_transfer_pl.with_columns(pl.col('amount').truediv(
    10**contract_settings[protocol_name]['decimals']).alias('amount'))
print('There are {} Transfer events'.format(len(dydx_transfer_pl)))
dydx_transfer_pl.head()

Loading Transfer events: 100%|██████████| 796672/796672 [00:01<00:00, 778621.14it/s]


There are 747372 Transfer events


blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,from,to,amount,timestamp
i64,str,str,str,i64,i64,str,str,str,f64,datetime[ns]
12809555,"""c651e83150a597d999c07d1e4689a8…","""364b2b2a4ba75661940d7838792b00…","""0x92d6c1e31e14520e676a687f0a93…",53,57,"""Transfer""","""0x0000000000000000000000000000…","""0x301df37d653b281af83a1ddf4464…",1.000000000000000119e9,2021-07-12 01:37:12
12824766,"""5866c5d6101cd4e25b83dea598ca21…","""389812942f89a9208e257361f79c01…","""0x92d6c1e31e14520e676a687f0a93…",107,61,"""Transfer""","""0x301df37d653b281af83a1ddf4464…","""0xeb74327ddd3f0d359321f06d84a9…",2.000000000000000000e0,2021-07-14 11:04:15
12824768,"""759dcde97cc407b574d6cd58fb32dd…","""e5d849dfc0db2b860649fbc9aafddc…","""0x92d6c1e31e14520e676a687f0a93…",116,81,"""Transfer""","""0x301df37d653b281af83a1ddf4464…","""0xb03414a51a625e8ce16d284e3494…",2.000000000000000000e0,2021-07-14 11:04:57
12824772,"""e6c2df6b0105be70229b2a2c393413…","""bc7aa7c67c48a32da5454506f58ae4…","""0x92d6c1e31e14520e676a687f0a93…",16,17,"""Transfer""","""0x301df37d653b281af83a1ddf4464…","""0x1eec5afab429859c46db6552bc97…",2.000000000000000000e0,2021-07-14 11:06:06
12824779,"""f556296d38de9e5068b8ded2a5f452…","""091d78572295d9c9cf7dc7d1fa8941…","""0x92d6c1e31e14520e676a687f0a93…",39,93,"""Transfer""","""0x301df37d653b281af83a1ddf4464…","""0x69112552fac655bb76a3e0ee7779…",2.000000000000000000e0,2021-07-14 11:06:54


In [21]:
protocol_name = 'dydx-merkle-distributor'
# Treasury Merkle Claim Proxy
dydx_address_to_ignore = "0x95eabb0248d013b9f59c5d5256ce11b0a8140b54"
dydx_claim_pl = load_claim(protocol_name, amount_column="amount",
                           event_claim="RewardsClaimed", account_column="account")
dydx_claim_pl = (
    dydx_claim_pl
    .filter(pl.col("account").ne(dydx_address_to_ignore))
    .with_columns(pl.col('amount').truediv(
        10**contract_settings[protocol_name]['decimals']).alias('amount'))
)
print('There are {} Claim events'.format(len(dydx_claim_pl)))
dydx_claim_pl.head()

Loading Claim events: 100%|██████████| 73115/73115 [00:00<00:00, 781927.61it/s]


There are 72870 Claim events


blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,account,amount,timestamp
i64,str,str,str,i64,i64,str,str,f64,datetime[ns]
13183164,"""361400ec679147ed4575ac2215de48…","""66eebcd18b681919affcf267100740…","""0x01d3348601968ab85b4bb0289790…",18,37,"""RewardsClaimed""","""0x54af6a1dbacfa9dddd72a7dd1b4f…",1.584922969314624952e-1,2021-09-08 05:24:48
13185813,"""751c7b32ba3ed77f59dc2776a910dc…","""bab2daf7004c944c5311039628208f…","""0x01d3348601968ab85b4bb0289790…",23,102,"""RewardsClaimed""","""0x8568e1a8082b442ae9be089a3b38…",1.178209769696859667e3,2021-09-08 15:00:53
13185813,"""f1664ae2da908ee8a71e88afe86d25…","""bab2daf7004c944c5311039628208f…","""0x01d3348601968ab85b4bb0289790…",156,242,"""RewardsClaimed""","""0x44acd0ff3be9fdfb932453c82b5d…",4.350810345703857820e3,2021-09-08 15:00:53
13185813,"""0f6a0ed8d5e6854709e12d57bb65b6…","""bab2daf7004c944c5311039628208f…","""0x01d3348601968ab85b4bb0289790…",158,254,"""RewardsClaimed""","""0xf7e39bfa90e8501fe2f2a1333e3e…",1.163646413569437755e3,2021-09-08 15:00:53
13185813,"""9087ed3f61e5d0f1d98a89ba35e982…","""bab2daf7004c944c5311039628208f…","""0x01d3348601968ab85b4bb0289790…",238,348,"""RewardsClaimed""","""0x85a60b688a8057683fd4b46f4268…",6.478038445727037470e3,2021-09-08 15:00:53


In [22]:
# dydx_claiming_address = "0x639192d54431f8c816368d3fb4107bc168d0e871"
# dydx_claim_pl = dydx_transfer_pl.filter(
#     pl.col("from").eq(dydx_claiming_address))
# print('There are {} Claim events'.format(len(dydx_claim_pl)))
# dydx_claim_pl.head()

In [23]:
file_path = os.path.realpath(os.path.join(
    path_data, "dydx_transfer.parquet"))
dydx_transfer_pl.write_parquet(file_path, compression="gzip")

file_path = os.path.realpath(os.path.join(path_data, "dydx_claim.parquet"))
dydx_claim_pl.write_parquet(file_path, compression="gzip")

## Lido


In [24]:
protocol_name = 'lido-token'
lido_transfer_pl = load_transfer(
    protocol_name, amount_column="_amount", sender="_from", receiver="_to")
lido_transfer_pl = lido_transfer_pl.with_columns(pl.col('amount').truediv(
    10**contract_settings[protocol_name]['decimals']).alias('amount'))
print('There are {} Transfer events'.format(len(lido_transfer_pl)))
lido_transfer_pl.head()

Loading Transfer events: 100%|██████████| 1227365/1227365 [00:01<00:00, 765460.67it/s]


There are 1107586 Transfer events


blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,from,to,amount,timestamp
i64,str,str,str,i64,i64,str,str,str,f64,datetime[ns]
11473276,"""06c10f98d32cdbe8633c967aabd52c…","""a73f4a3d175aa7b81822a228788cd9…","""0x5a98fcbea516cf06857215779fd8…",84,124,"""Transfer""","""0x0000000000000000000000000000…","""0xf73a1260d222f447210581ddf212…",2.364025000000000000e8,2020-12-17 22:12:05
11473276,"""06c10f98d32cdbe8633c967aabd52c…","""a73f4a3d175aa7b81822a228788cd9…","""0x5a98fcbea516cf06857215779fd8…",84,125,"""Transfer""","""0xf73a1260d222f447210581ddf212…","""0x9bb75183646e2a0dc855498bacd7…",2.000000000000000373e7,2020-12-17 22:12:05
11473276,"""06c10f98d32cdbe8633c967aabd52c…","""a73f4a3d175aa7b81822a228788cd9…","""0x5a98fcbea516cf06857215779fd8…",84,127,"""Transfer""","""0xf73a1260d222f447210581ddf212…","""0x0f89d54b02ca570de82f770d33c7…",2.500000000000000373e7,2020-12-17 22:12:05
11473276,"""06c10f98d32cdbe8633c967aabd52c…","""a73f4a3d175aa7b81822a228788cd9…","""0x5a98fcbea516cf06857215779fd8…",84,129,"""Transfer""","""0xf73a1260d222f447210581ddf212…","""0xe49f68b9a01d437b0b7ea416376a…",2.282000000000000000e6,2020-12-17 22:12:05
11473276,"""06c10f98d32cdbe8633c967aabd52c…","""a73f4a3d175aa7b81822a228788cd9…","""0x5a98fcbea516cf06857215779fd8…",84,131,"""Transfer""","""0xf73a1260d222f447210581ddf212…","""0xb842afd82d940ff5d8f6ef339957…",1.771800000000000000e7,2020-12-17 22:12:05


In [25]:
lido_early_stakers_airdrop_address = "0x4b3edb22952fb4a70140e39fb1add05a6b49622b"
lido_1inch_liquidity_farming_address = "0xdb46c277da1599390eab394327602889e9546296"
lido_claim_pl = lido_transfer_pl.filter(
    pl.col("from").eq(lido_early_stakers_airdrop_address) | pl.col('from').eq(lido_1inch_liquidity_farming_address))
print('There are {} Claim events'.format(len(lido_claim_pl)))
lido_claim_pl.head()

There are 516 Claim events


blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,from,to,amount,timestamp
i64,str,str,str,i64,i64,str,str,str,f64,datetime[ns]
11596109,"""730f238b6aba4c92c74a2db53c6623…","""4d7749b8b109b11ab10496ccac4c39…","""0x5a98fcbea516cf06857215779fd8…",29,41,"""Transfer""","""0x4b3edb22952fb4a70140e39fb1ad…","""0x55bc991b2edf3ddb4c520b222be4…",4.892206212239684646e2,2021-01-05 18:16:51
11596110,"""aa53383eebea4dab91ef42c38c3518…","""dbb15f961a47545b361bfad9f769d3…","""0x5a98fcbea516cf06857215779fd8…",77,217,"""Transfer""","""0x4b3edb22952fb4a70140e39fb1ad…","""0x44ea6f4805b92b14951c109fed1e…",1.242263015234035947e3,2021-01-05 18:17:02
11596114,"""0d1f4d777828bb527f5f8c149de251…","""98e5ca581aa334a27b8ba24cb0d081…","""0x5a98fcbea516cf06857215779fd8…",132,256,"""Transfer""","""0x4b3edb22952fb4a70140e39fb1ad…","""0x912392a282500bb395b3af9a091b…",1.381364913659388776e3,2021-01-05 18:18:01
11596122,"""c7a454747491dc04eb1738005d0420…","""2c5324a40a7f116466fa1caa2a0e6c…","""0x5a98fcbea516cf06857215779fd8…",0,0,"""Transfer""","""0x4b3edb22952fb4a70140e39fb1ad…","""0xf30b321970b3a4bba00d068284f9…",1.284346614560077455e2,2021-01-05 18:19:48
11596125,"""9b05112aaba57630e9f44d52ce1c5f…","""866da2f5409a9ca9ce1441b846fc31…","""0x5a98fcbea516cf06857215779fd8…",58,133,"""Transfer""","""0x4b3edb22952fb4a70140e39fb1ad…","""0x41e37ddde5c80ee80cb9c468c0ed…",2.555293740455965121e2,2021-01-05 18:20:16


In [26]:
file_path = os.path.realpath(os.path.join(
    path_data, "lido_transfer.parquet"))
lido_transfer_pl.write_parquet(file_path, compression="gzip")

file_path = os.path.realpath(os.path.join(path_data, "lido_claim.parquet"))
lido_claim_pl.write_parquet(file_path, compression="gzip")

## TornadoCash


In [27]:
protocol_name = 'tornadocash'
tornado_transfer_pl = load_transfer(
    protocol_name, amount_column="value")
tornado_transfer_pl = tornado_transfer_pl.with_columns(pl.col('amount').truediv(
    10**contract_settings[protocol_name]['decimals']).alias('amount'))
print('There are {} Transfer events'.format(len(tornado_transfer_pl)))
tornado_transfer_pl.head()

Loading Transfer events: 100%|██████████| 321928/321928 [00:00<00:00, 805310.85it/s]


There are 278455 Transfer events


blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,from,to,amount,timestamp
i64,str,str,str,i64,i64,str,str,str,f64,datetime[ns]
11474599,"""fbe35150a2bd11d2fa1b7ebd25a960…","""de33e4bb75c4c10b452db4d4bebff6…","""0x77777feddddffc19ff86db637967…",22,17,"""Transfer""","""0x0000000000000000000000000000…","""0x3efa30704d2b8bbac82130723037…",5.000000000000000000e5,2020-12-18 03:00:37
11474599,"""fbe35150a2bd11d2fa1b7ebd25a960…","""de33e4bb75c4c10b452db4d4bebff6…","""0x77777feddddffc19ff86db637967…",22,18,"""Transfer""","""0x0000000000000000000000000000…","""0x5cab7692d4e94096462119ab7bf5…",1.000000000000000000e6,2020-12-18 03:00:37
11474599,"""fbe35150a2bd11d2fa1b7ebd25a960…","""de33e4bb75c4c10b452db4d4bebff6…","""0x77777feddddffc19ff86db637967…",22,19,"""Transfer""","""0x0000000000000000000000000000…","""0x179f48c78f57a3a78f0608cc9197…",5.500000000000000931e6,2020-12-18 03:00:37
11474599,"""fbe35150a2bd11d2fa1b7ebd25a960…","""de33e4bb75c4c10b452db4d4bebff6…","""0x77777feddddffc19ff86db637967…",22,20,"""Transfer""","""0x0000000000000000000000000000…","""0x5f48c2a71b2cc96e3f0ccae4e393…",8.224070000000001164e5,2020-12-18 03:00:37
11474599,"""fbe35150a2bd11d2fa1b7ebd25a960…","""de33e4bb75c4c10b452db4d4bebff6…","""0x77777feddddffc19ff86db637967…",22,21,"""Transfer""","""0x0000000000000000000000000000…","""0x00d5ec4cdf59374b2a47e842b799…",8.224070000000001164e5,2020-12-18 03:00:37


In [28]:
tornado_claiming_address = "0x3efa30704d2b8bbac821307230376556cf8cc39e"
tornado_claim_pl = tornado_transfer_pl.filter(
    pl.col("from").eq(tornado_claiming_address))
print('There are {} Claim events'.format(len(tornado_claim_pl)))
tornado_claim_pl.head()

There are 5474 Claim events


blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,from,to,amount,timestamp
i64,str,str,str,i64,i64,str,str,str,f64,datetime[ns]
11474827,"""a4e086ed94ef306c46355a8d7b11b3…","""426394afbcba58706504b9758f1f7e…","""0x77777feddddffc19ff86db637967…",60,80,"""Transfer""","""0x3efa30704d2b8bbac82130723037…","""0x3238b53a910b69f5dbdb31786613…",1.626185898769037976e2,2020-12-18 03:52:06
11475464,"""8292c7b8393321097dfb1fbaa06449…","""bd997e3440822fd74edc66914ae145…","""0x77777feddddffc19ff86db637967…",119,317,"""Transfer""","""0x3efa30704d2b8bbac82130723037…","""0xd5bb23d16ba5a13a91d60dd5f392…",2.543934624398110955e1,2020-12-18 06:13:48
11475638,"""2b6906f931a20743e0b2c75cdb06ef…","""c951b3aa1e7a67a3373114c7e6da60…","""0x77777feddddffc19ff86db637967…",143,170,"""Transfer""","""0x3efa30704d2b8bbac82130723037…","""0x84ecb387395a1be65e133c75ff9e…",2.021910030772594524e1,2020-12-18 06:49:55
11475710,"""a1e70fff06d0aaf23218825f1d286b…","""a7ec8b00a5f8e14c7f764e46a6bda1…","""0x77777feddddffc19ff86db637967…",174,210,"""Transfer""","""0x3efa30704d2b8bbac82130723037…","""0x531c036b76509604817222ab2905…",1.527050300324847143e1,2020-12-18 07:05:44
11475762,"""b03ed794c1ecd90d13fcae14c92bb2…","""87b5cbd0c2408a7e467885001e0683…","""0x77777feddddffc19ff86db637967…",109,94,"""Transfer""","""0x3efa30704d2b8bbac82130723037…","""0x7d8d401a314f8b1bb0f5443d593e…",2.408670566365820420e1,2020-12-18 07:15:51


In [29]:
file_path = os.path.realpath(os.path.join(
    path_data, "tornado_transfer.parquet"))
tornado_transfer_pl.write_parquet(file_path, compression="gzip")

file_path = os.path.realpath(os.path.join(path_data, "tornado_claim.parquet"))
tornado_claim_pl.write_parquet(file_path, compression="gzip")

## Compound


In [30]:
protocol_name = 'compound'
compound_transfer_pl = load_transfer(
    protocol_name, amount_column="amount")
compound_transfer_pl = compound_transfer_pl.with_columns(pl.col('amount').truediv(
    10**contract_settings[protocol_name]['decimals']).alias('amount'))
print('There are {} Transfer events'.format(len(compound_transfer_pl)))
compound_transfer_pl.head()

Loading Transfer events: 100%|██████████| 2311077/2311077 [00:02<00:00, 805495.77it/s]


There are 2242826 Transfer events


blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,from,to,amount,timestamp
i64,str,str,str,i64,i64,str,str,str,f64,datetime[ns]
9601359,"""e87715364f1733c893b4dca5c8e932…","""4bd3833532f62c8a5a8d261c608994…","""0xc00e94cb662c3520282e6f571721…",17,19,"""Transfer""","""0x0000000000000000000000000000…","""0x1449e0687810bddd356ae6dd8778…",1.000000000000000186e7,2020-03-04 00:08:55
9619917,"""54b2f30e2f608c2857499b089da43a…","""e9242498a30035fc33083d95b47425…","""0xc00e94cb662c3520282e6f571721…",17,16,"""Transfer""","""0x1449e0687810bddd356ae6dd8778…","""0x49964dca5c033b1ac8a58d550576…",1.000000000000000000e0,2020-03-06 20:45:00
9638547,"""45cec956c648054886dffbec746bb1…","""786a86e598f817a331acf678049ebd…","""0xc00e94cb662c3520282e6f571721…",150,148,"""Transfer""","""0x49964dca5c033b1ac8a58d550576…","""0x1449e0687810bddd356ae6dd8778…",1.000000000000000000e0,2020-03-09 17:22:24
9639140,"""860c08440a96b2be4b3afcab50af1f…","""01d118dcd83da3d4bfe75e98a2e7f6…","""0xc00e94cb662c3520282e6f571721…",53,71,"""Transfer""","""0x1449e0687810bddd356ae6dd8778…","""0x49964dca5c033b1ac8a58d550576…",1.000000000000000000e0,2020-03-09 19:29:51
9640293,"""59fe0334e75f6a5ff0d083f8e1a488…","""2815012ebe951e6e877b582a5eaa5f…","""0xc00e94cb662c3520282e6f571721…",27,48,"""Transfer""","""0x49964dca5c033b1ac8a58d550576…","""0x73905a4d38b749be7d5674418ef3…",1.000000000000000000e0,2020-03-09 23:47:04


## WorldCoin

It seems their airdrop was conducted on Optimism. Therefore, we'll leave it out of our analysis for now.


In [31]:
protocol_name = 'worldcoin'
worldcoin_transfer_pl = load_transfer(
    protocol_name, amount_column="value")
worldcoin_transfer_pl = worldcoin_transfer_pl.with_columns(pl.col('amount').truediv(
    10**contract_settings[protocol_name]['decimals']).alias('amount'))
print('There are {} Transfer events'.format(len(worldcoin_transfer_pl)))
worldcoin_transfer_pl.head()

Loading Transfer events: 100%|██████████| 420451/420451 [00:00<00:00, 799785.26it/s]


There are 294481 Transfer events


blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,from,to,amount,timestamp
i64,str,str,str,i64,i64,str,str,str,f64,datetime[ns]
17714705,"""e884a49cc0bad8a49bb6149dd7c185…","""6633751c1cd5bbafa4ee87a3c1363d…","""0x163f8c2467924be0ae7b5347228c…",38,427,"""Transfer""","""0x0000000000000000000000000000…","""0x8155ef75b9ccaff88a6d247b2bea…",1.656555000000000238e9,2023-07-17 18:16:47
17714705,"""e884a49cc0bad8a49bb6149dd7c185…","""6633751c1cd5bbafa4ee87a3c1363d…","""0x163f8c2467924be0ae7b5347228c…",38,428,"""Transfer""","""0x0000000000000000000000000000…","""0xfb7f100ed6793fcd63e6e9f201be…",1.650000000000000186e7,2023-07-17 18:16:47
17714705,"""e884a49cc0bad8a49bb6149dd7c185…","""6633751c1cd5bbafa4ee87a3c1363d…","""0x163f8c2467924be0ae7b5347228c…",38,429,"""Transfer""","""0x0000000000000000000000000000…","""0xfb7f100ed6793fcd63e6e9f201be…",1.000000000000000186e7,2023-07-17 18:16:47
17714705,"""e884a49cc0bad8a49bb6149dd7c185…","""6633751c1cd5bbafa4ee87a3c1363d…","""0x163f8c2467924be0ae7b5347228c…",38,430,"""Transfer""","""0x0000000000000000000000000000…","""0xfb7f100ed6793fcd63e6e9f201be…",4.000000000000000000e6,2023-07-17 18:16:47
17714705,"""e884a49cc0bad8a49bb6149dd7c185…","""6633751c1cd5bbafa4ee87a3c1363d…","""0x163f8c2467924be0ae7b5347228c…",38,431,"""Transfer""","""0x0000000000000000000000000000…","""0xfb7f100ed6793fcd63e6e9f201be…",2.500000000000000466e6,2023-07-17 18:16:47


## Sushiswap


In [32]:
protocol_name = 'sushi'
data = load_dataset(protocol_name)
sushi_transfer_df = Utils.transfer_to_dataframe(
    data['Transfer'], amount_colum="value", dataframe_func=pd.DataFrame)
sushi_transfer_df['amount'] = sushi_transfer_df['amount'] / \
    10**contract_settings[protocol_name]['decimals']
sushi_transfer_pl = pl.DataFrame(sushi_transfer_df)

file_path = os.path.realpath(os.path.join(path_data, "blocks.parquet.gz"))
blocks_pl = (pl.scan_parquet(file_path)
             .select(['number', 'timestamp'])).collect()

sushi_transfer_pl = (sushi_transfer_pl
                     .join(blocks_pl, left_on='blockNumber', right_on='number')
                     .sort(['blockNumber', 'transactionIndex', 'logIndex'])
                     )

sushi_transfer_pl.head()

Loading Transfer events: 100%|██████████| 3791064/3791064 [00:04<00:00, 816966.97it/s]


blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,from,to,amount,timestamp
i64,str,str,str,i64,i64,str,str,str,f64,datetime[ns]
10737828,"""19e0283e1f5d5e8e78fbf025f11ed6…","""a0960869b7e7e16ab70604f534cc48…","""0x6b3595068778dd592e39a122f4f5…",77,62,"""Transfer""","""0xc2edad668740f1aa35e4d8f227fb…","""0xf83d9f34ba9fea4e0b6c55d56187…",0.000000000000000000e0,2020-08-26 18:46:51
10738166,"""5421ac6ff434c71994bd6654114c26…","""677d54cfb31548f31ad7cb104a014b…","""0x6b3595068778dd592e39a122f4f5…",159,228,"""Transfer""","""0xc2edad668740f1aa35e4d8f227fb…","""0x1183634e867ff81231cbd0be2438…",0.000000000000000000e0,2020-08-26 19:57:08
10739217,"""5b9f121bb2db655d7bf4d3795cb7b1…","""9201cfb2090fb43569d2659d024876…","""0x6b3595068778dd592e39a122f4f5…",80,173,"""Transfer""","""0xc2edad668740f1aa35e4d8f227fb…","""0x4c8397f58d62e3b8fd1fa47ca897…",0.000000000000000000e0,2020-08-26 23:51:41
10739805,"""9777592851d16cc6768ea8492ea70b…","""a1a4178a2a89f64bd3d5a11b76d21c…","""0x6b3595068778dd592e39a122f4f5…",159,195,"""Transfer""","""0xc2edad668740f1aa35e4d8f227fb…","""0x1183634e867ff81231cbd0be2438…",0.000000000000000000e0,2020-08-27 02:07:38
10740034,"""3bf0dd3907db7e8fc0ca27c027ccd9…","""442b87ced62d09e7ae83859049bd8c…","""0x6b3595068778dd592e39a122f4f5…",99,36,"""Transfer""","""0xc2edad668740f1aa35e4d8f227fb…","""0xc50eaaee441566b4ef4f1e3270d1…",0.000000000000000000e0,2020-08-27 02:58:20


In [33]:
sushi_transfer_pl.filter(pl.col('from').ne('0x0000000000000000000000000000000000000000'))[
    'to'].value_counts().sort(by='count', descending=True).rows()

[('0x795065dcc9f64b5614c407a6efdc400da6221fb0', 307950),
 ('0xce84867c3c02b05dc570d0135103d3fb9cc19433', 158766),
 ('0x8798249c2e607446efb7ad49ec89dd1865ff4272', 103492),
 ('0x56178a0d5f301baf6cf3e1cd53d9863437345bf9', 58276),
 ('0x28c6c06298d514db089934071355e5743bf21d60', 34842),
 ('0xe93381fb4c4f14bda253907b18fad305d799241a', 33477),
 ('0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be', 33070),
 ('0x74de5d4fcbf63e00296fd95d33236b9794016631', 22286),
 ('0x95a9bd206ae52c4ba8eecfc93d18eacdd41c88cc', 21311),
 ('0xac844b604d6c600fbe55c4383a6d87920b46a160', 20788),
 ('0x6cc5f688a315f3dc28a7781717a9a798a59fda7b', 20473),
 ('0x2faf487a4414fe77e2327f0bf4ae2a264a776ad2', 17370),
 ('0xfa103c21ea2df71dfb92b0652f8b1d795e51cdef', 15503),
 ('0x0000006daea1723962647b7e189d311d757fb793', 14397),
 ('0x73a6a761fe483ba19debb8f56ac5bbf14c0cdad1', 13234),
 ('0xd9e1ce17f2641f24ae83637ab66a2cca9c378b9f', 12566),
 ('0xa69babef1ca67a37ffaf7a485dfff3382056e78c', 10524),
 ('0x7a250d5630b4cf539739df2c5dacb4c659f2488d

In [34]:
sushi_transfer_pl.head(100_000).filter(
    pl.col('amount').ne(0))['from'].value_counts().sort(by='count', descending=True).rows()

[('0x0000000000000000000000000000000000000000', 46762),
 ('0xc2edad668740f1aa35e4d8f227fb8e17dca888cd', 22052),
 ('0xce84867c3c02b05dc570d0135103d3fb9cc19433', 9592),
 ('0x80c5e6908368cb9db503ba968d7ec5a565bfb389', 1113),
 ('0x7a250d5630b4cf539739df2c5dacb4c659f2488d', 1079),
 ('0x1f8a6d8185b76f5ac08f335610a2390dab3225a8', 225),
 ('0x11111254369792b2ca5d084ab5eea397ca8fa48b', 162),
 ('0x6cb2291a3c3794fca0f5b6e34a8e6ea7933ca667', 158),
 ('0x480ea104ff7063ed0af41c98d8ef2457afe2a41c', 157),
 ('0x0d0707963952f2fba59dd06f2b425ace40b492fe', 142),
 ('0xab5ffdcbe8ae4c0c305aa5017f06141974cae591', 107),
 ('0x7fe2153de0006d76c85cc04c8ea10bf4546c879e', 107),
 ('0x343e3a490c9251dc0eaa81da146ba6abe6c78b2d', 93),
 ('0x693c188e40f760ecf00d2946ef45260b84fbc43e', 80),
 ('0x9769e91921fe42e48eb6a5fefb9f97aacb524ef0', 77),
 ('0xff18e262d17f0bb176417d30f4c50c7d6416963d', 70),
 ('0xcfea022285f87080535c07769ac2ef7aef3308ce', 69),
 ('0x47a5b6c45561279ef8f4088031d2c1801bbdc385', 66),
 ('0xbee81b34196e68cf62421d

In [35]:
sushi_transfer_pl.head(100_000)['from'].value_counts().sort(
    by="count", descending=True).rows()

[('0x0000000000000000000000000000000000000000', 46762),
 ('0xc2edad668740f1aa35e4d8f227fb8e17dca888cd', 22486),
 ('0xce84867c3c02b05dc570d0135103d3fb9cc19433', 9592),
 ('0x80c5e6908368cb9db503ba968d7ec5a565bfb389', 1113),
 ('0x7a250d5630b4cf539739df2c5dacb4c659f2488d', 1079),
 ('0x1f8a6d8185b76f5ac08f335610a2390dab3225a8', 225),
 ('0x11111254369792b2ca5d084ab5eea397ca8fa48b', 162),
 ('0x6cb2291a3c3794fca0f5b6e34a8e6ea7933ca667', 158),
 ('0x480ea104ff7063ed0af41c98d8ef2457afe2a41c', 157),
 ('0x0d0707963952f2fba59dd06f2b425ace40b492fe', 142),
 ('0x7fe2153de0006d76c85cc04c8ea10bf4546c879e', 107),
 ('0xab5ffdcbe8ae4c0c305aa5017f06141974cae591', 107),
 ('0x343e3a490c9251dc0eaa81da146ba6abe6c78b2d', 93),
 ('0x693c188e40f760ecf00d2946ef45260b84fbc43e', 80),
 ('0x9769e91921fe42e48eb6a5fefb9f97aacb524ef0', 77),
 ('0xff18e262d17f0bb176417d30f4c50c7d6416963d', 70),
 ('0xcfea022285f87080535c07769ac2ef7aef3308ce', 69),
 ('0x47a5b6c45561279ef8f4088031d2c1801bbdc385', 66),
 ('0xbee81b34196e68cf62421d

## Gemstone


In [36]:
gemstone_decimals = Utils.load_contract_settings(
    chain="zksync")['gemstone-token']['decimals']

In [37]:
import pandas as pd
file_dir = os.path.realpath(os.path.join(
    path_data, "gemstone_transfer.csv.gz"))

gemstone_df = pd.read_csv(file_dir, sep=";")

gemstone_transfer_pl = pl.DataFrame(gemstone_df)

gemstone_transfer_pl = (gemstone_transfer_pl
                        .with_columns([pl.col('timestamp').str.to_datetime(),
                                       pl.col('amount').cast(pl.Int128).truediv(10**gemstone_decimals)])

                        )

file_dir = os.path.realpath(os.path.join(
    path_data, "gemstone_transfer.parquet"))
gemstone_transfer_pl.write_parquet(file_dir)

In [38]:
file_dir = os.path.realpath(os.path.join(
    path_data, "gemstone_claim.csv.gz"))

gemstone_claim_df = pd.read_csv(file_dir, sep=";")

gemstone_claim_pl = pl.DataFrame(gemstone_claim_df)

gemstone_claim_pl = (gemstone_claim_pl
                     .with_columns([pl.col('timestamp').str.to_datetime(),
                                    pl.col('amount').cast(pl.Int128).truediv(10**gemstone_decimals)])
                     .rename({"user": "account"})

                     )

file_dir = os.path.realpath(os.path.join(
    path_data, "gemstone_claim.parquet"))
gemstone_claim_pl.write_parquet(file_dir)
gemstone_claim_pl.head()

blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,l1BatchNumber,logType,removed,transactionLogIndex,eventType,account,amount,timestamp
i64,str,str,str,i64,i64,i64,f64,bool,i64,str,str,f64,datetime[μs]
9375492,"""0x51af8cf28865e15bf9b4477cca97…","""0x83df759223617c84cf807396c577…","""0xdea197d9920a7472050013ad7e8d…",4,21,125859,null,false,21,"""Airdrop""","""0xc4d0ed974291ac94d6a3e1bf24eb…",2.600000000000000000e12,2023-07-23 21:50:55
9375497,"""0x922feb33a556d7b30216b3436af5…","""0x9406fe83b04881c9e0e43c9ed91c…","""0xdea197d9920a7472050013ad7e8d…",0,2,125859,null,false,2,"""Airdrop""","""0x5d500c14a1dcaf2842b60aa3a6ec…",0.000000000000000000e0,2023-07-23 21:50:55
9405853,"""0xc2b30ed87009983a595921b23633…","""0x84eddbb62852998a69331ac3ad81…","""0xdea197d9920a7472050013ad7e8d…",13,69,126502,null,false,69,"""Airdrop""","""0xebbb96033d8a1ad9c82c4a46161d…",0.000000000000000000e0,2023-07-24 06:22:34
9408062,"""0xc779972f9f0ea28a65bf2e8feb21…","""0xdc82c8e9da70912928c2a42f44c0…","""0xdea197d9920a7472050013ad7e8d…",14,98,126598,null,false,98,"""Airdrop""","""0xcad49f762b93e57a849a22c257d0…",2.558000000000000488e12,2023-07-24 06:59:45
9408086,"""0xc874d11506dae97aa206ed683e97…","""0x30f95ad0ce09652d65234c492649…","""0xdea197d9920a7472050013ad7e8d…",7,35,126599,null,false,35,"""Airdrop""","""0x9943320f763aa353ec9ec974d826…",2.518000000000000000e12,2023-07-24 07:00:09


## Arbitrum


In [39]:
from utils import events_dict, get_event_name

In [40]:
transfer_event = "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef"
can_claim_event = "0x87aeeb9eda09a064caef63d00f62c15063631980bfc422ad7dd30c8a79f0cbb7"
has_claimed_event = "0x8629b200ebe43db58ad688b85131d53251f3f3be4c14933b4641aeebacf1c08c"
arb_decimals = Utils.load_contract_settings(
    chain="arbitrum")['arbitrum-token']['decimals']

In [41]:
column_order = ['block_number', 'block_hash', 'transaction_hash',
                'transaction_index', 'log_index', 'address', 'data', 'topics', 'block_timestamp']
schema = {'log_index': pl.Int64,
          'transaction_hash': pl.String,
          'transaction_index': pl.Int64,
          'address': pl.String,
          'data': pl.String,
          'topics': pl.String,
          'block_timestamp': pl.String,
          'block_number': pl.Int64,
          'block_hash': pl.String}


def convert_arbitrum_to_parquet():
    path_dir = os.path.realpath(os.path.join(
        path_data, "arbitrum"))
    filenames = [filename for filename in os.listdir(
        path_dir) if filename.startswith('arb_') and filename.endswith('.csv.gz')]

    for filename in tqdm(filenames):
        df = pd.read_csv(path_dir+filename, compression='gzip')
        new_filename = filename.replace('.csv.gz', '.parquet.gz')
        df = pl.DataFrame(df[column_order], schema=schema)
        df.write_parquet(
            path_dir+new_filename, compression='gzip')

In [42]:
# Parse dataframe
path_arbitrum = os.path.realpath(os.path.join(
    path_data, "arbitrum"))
file_path = os.path.realpath(os.path.join(path_arbitrum, "arb_*.parquet.gz"))
arbitrum_transfer_pl = (
    pl.scan_parquet(file_path)
    .with_columns(
        pl.col("topics").str.replace_all('\n', ',').map_elements(eval, pl.List(str)))
    .with_columns(pl.col('topics').list.first().alias('topics_0'))
    .filter(pl.col('topics_0').eq(transfer_event))
    .with_columns(pl.format('0x{}', pl.col('topics').list.get(1).str.slice(-40)
                            ).str.to_lowercase().alias('from'),
                  pl.format('0x{}', pl.col('topics').list.get(2).str.slice(-40)
                            ).str.to_lowercase().alias('to'),
                  pl.col('data').str.replace('0x', '0x0').map_elements(
        Utils.parse_amount, pl.Float64).truediv(10**arb_decimals).alias('amount'),
        pl.col('block_timestamp').str.to_datetime().alias('timestamp')
    )
    .sort(by=['block_number', 'transaction_index', 'log_index'], descending=False)
    .select([pl.col("block_number").alias("blockNumber"),
             pl.col("transaction_hash").alias("transactionHash"),
             pl.col("block_hash").alias("blockHash"),
             pl.col('address'),
             pl.col("transaction_index").alias("transactionIndex"),
             pl.col("log_index").alias("logIndex"),
             pl.col("from"), pl.col("to"),
             pl.col("amount"), pl.col("timestamp"),
             ])

).collect(new_streaming=True)

arbitrum_transfer_pl.head()

blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,from,to,amount,timestamp
i64,str,str,str,i64,i64,str,str,f64,"datetime[μs, UTC]"
70398215,"""0x9cdbb4672b549c26d97cac29f9cd…","""0xc7a696bb947210c89bd9225277e9…","""0x912ce59144191c1204e64559fe82…",1,19,"""0x0000000000000000000000000000…","""0x2b9acfd85440b7828db8e54694ee…",1.000000000000000000e10,2023-03-16 12:08:28 UTC
70506540,"""0xc815f5fae3a872f6fa01f734cb76…","""0x52afee55f30d8530748dc763fc0e…","""0x912ce59144191c1204e64559fe82…",3,2,"""0x2b9acfd85440b7828db8e54694ee…","""0xf3fc178157fb3c87548baa86f9d2…",3.527046079000000477e9,2023-03-16 19:36:42 UTC
70506548,"""0x9dd86c966adf49238f88b17e5ce8…","""0x388d2550781d158707366e09e90c…","""0x912ce59144191c1204e64559fe82…",1,0,"""0x2b9acfd85440b7828db8e54694ee…","""0xc24e24383120669512a336fb3b5b…",7.500000000000000000e8,2023-03-16 19:36:44 UTC
70506569,"""0x390910c3692b22cec48673b3bfa6…","""0x34db953c6003b77b2cec25304908…","""0x912ce59144191c1204e64559fe82…",1,0,"""0x2b9acfd85440b7828db8e54694ee…","""0x148440e028565db5f790fa3fbdbb…",2.693961299000000477e9,2023-03-16 19:36:50 UTC
70506593,"""0xa301736bf5115fb13389ce2eccf8…","""0xb6f232044a51c2ea0644bb49d658…","""0x912ce59144191c1204e64559fe82…",2,0,"""0x2b9acfd85440b7828db8e54694ee…","""0xdca2bd62cdcb5f742dd3f97e820d…",1.128340000000000000e8,2023-03-16 19:36:56 UTC


In [43]:
file_path = os.path.realpath(os.path.join(
    path_arbitrum, "arbitrum_logs.parquet"))
arbitrum_has_claimed_pl = (
    pl.scan_parquet(file_path)
    .drop_nulls()
    .with_columns(
        pl.col("topics").str.replace_all('\n', ',').map_elements(eval, pl.List(str)))
    .with_columns(pl.col('topics').list.first().alias('topics_0'))
    .filter(pl.col('topics_0').eq(has_claimed_event))
    .with_columns(
        pl.format('0x{}', pl.col('topics').list.get(1).str.slice(-40)
                  ).str.to_lowercase().alias('account'),
        pl.col('data').str.replace('0x', '0x0').map_elements(
            Utils.parse_amount, pl.Float64).truediv(10**arb_decimals).alias('amount'),
        pl.col('block_timestamp').str.to_datetime().alias('timestamp')
    )
    .sort(by=['block_number', 'transaction_index', 'log_index'], descending=False)
    .select([
        pl.col("block_number").alias("blockNumber"),
        pl.col("transaction_hash").alias("transactionHash"),
        pl.col("block_hash").alias("blockHash"),
        pl.col('address'),
        pl.col("transaction_index").alias("transactionIndex"),
        pl.col("log_index").alias("logIndex"),
        pl.col("account"),
        pl.col("amount"),
        pl.col("timestamp"),
    ])

).collect(new_streaming=True)

arbitrum_has_claimed_pl.head()

blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,account,amount,timestamp
i64,str,str,str,i64,i64,str,f64,"datetime[μs, UTC]"
72829612,"""0x324f2ef46287a4f51a70e2a17ecd…","""0x9d20099f9d7e10987a19756780d1…","""0x67a24ce4321ab3af51c2d0a4801c…",1,2,"""0x27a1b27b2e8bfdd57a3049415dcf…",1.625000000000000227e3,2023-03-23 13:01:22 UTC
72829623,"""0x729e9b4a39d9e5026ec9b9312683…","""0x0bcd46818b8b34140c3038f57cfa…","""0x67a24ce4321ab3af51c2d0a4801c…",1,2,"""0x596aca03d432e0b869494c5e8258…",8.750000000000001137e2,2023-03-23 13:01:28 UTC
72829625,"""0x286a8e0045c41d70549cae998e56…","""0x6ce3b6176babbdbdc17dc2dc15d8…","""0x67a24ce4321ab3af51c2d0a4801c…",1,2,"""0x8d5614611aa239c680b4ff33a559…",1.125000000000000000e3,2023-03-23 13:01:34 UTC
72829626,"""0xc1c0b198a0fbd2d97579eff45c4a…","""0x9bfa53b9fd7f6769b4d875b3d62e…","""0x67a24ce4321ab3af51c2d0a4801c…",1,2,"""0x87cd88c7598b978116f20d62a12f…",8.750000000000001137e2,2023-03-23 13:01:34 UTC
72829626,"""0x62ddfe5b2375a2796097101f47d5…","""0x9bfa53b9fd7f6769b4d875b3d62e…","""0x67a24ce4321ab3af51c2d0a4801c…",2,5,"""0x0d2643aeb398e381612c4052c783…",1.625000000000000227e3,2023-03-23 13:01:34 UTC


In [44]:
file_path = os.path.realpath(os.path.join(
    path_arbitrum, "arbitrum_logs.parquet"))
arbitrum_can_claim_pl = (
    pl.scan_parquet(file_path)
    .drop_nulls()
    .with_columns(
        pl.col("topics").str.replace_all('\n', ',').map_elements(eval, pl.List(str)))
    .with_columns(pl.col('topics').list.first().alias('topics_0'))
    .filter(pl.col('topics_0').eq(can_claim_event))
    .with_columns(
        pl.format('0x{}', pl.col('topics').list.get(1).str.slice(-40)
                  ).str.to_lowercase().alias('account'),
        pl.col('data').str.replace('0x', '0x0').map_elements(
            Utils.parse_amount, pl.Float64).truediv(10**arb_decimals).alias('amount'),
        pl.col('block_timestamp').str.to_datetime().alias('timestamp')
    )
    .sort(by=['block_number', 'transaction_index', 'log_index'], descending=False)
    .select([
        pl.col("block_number").alias("blockNumber"),
        pl.col("transaction_hash").alias("transactionHash"),
        pl.col("block_hash").alias("blockHash"),
        pl.col('address'),
        pl.col("transaction_index").alias("transactionIndex"),
        pl.col("log_index").alias("logIndex"),
        pl.col("account"),
        pl.col("amount"),
        pl.col("timestamp"),
    ])

).collect(new_streaming=True)

arbitrum_can_claim_pl.head()

blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,account,amount,timestamp
i64,str,str,str,i64,i64,str,f64,"datetime[μs, UTC]"
70506697,"""0xa35d0da45119d9e8e6237cf257c1…","""0x5e9666aef3c4defac03ce7c2f359…","""0x67a24ce4321ab3af51c2d0a4801c…",2,3,"""0xb284bbf1864cbcfb2dff29671ff0…",6.250000000000000000e2,2023-03-16 19:37:23 UTC
70506697,"""0xa35d0da45119d9e8e6237cf257c1…","""0x5e9666aef3c4defac03ce7c2f359…","""0x67a24ce4321ab3af51c2d0a4801c…",2,4,"""0xf7b04982840d90a7634f41851815…",6.250000000000000000e2,2023-03-16 19:37:23 UTC
70506697,"""0xa35d0da45119d9e8e6237cf257c1…","""0x5e9666aef3c4defac03ce7c2f359…","""0x67a24ce4321ab3af51c2d0a4801c…",2,5,"""0x37b77b94ca23046f04a7fdd1af1a…",6.250000000000000000e2,2023-03-16 19:37:23 UTC
70506697,"""0xa35d0da45119d9e8e6237cf257c1…","""0x5e9666aef3c4defac03ce7c2f359…","""0x67a24ce4321ab3af51c2d0a4801c…",2,6,"""0x2686166054fd7de34872e1dc55aa…",6.250000000000000000e2,2023-03-16 19:37:23 UTC
70506697,"""0xa35d0da45119d9e8e6237cf257c1…","""0x5e9666aef3c4defac03ce7c2f359…","""0x67a24ce4321ab3af51c2d0a4801c…",2,7,"""0x0a988e9317bf7b73a4390522a995…",6.250000000000000000e2,2023-03-16 19:37:23 UTC


In [45]:
# Distribution of events
file_path = os.path.realpath(os.path.join(
    path_arbitrum, "arbitrum_logs.parquet"))
(
    pl.scan_parquet(file_path)
    .drop_nulls()
    .with_columns(
        pl.col("topics").str.replace_all('\n', ',').map_elements(eval, pl.List(str)))
    .with_columns(pl.col('topics').list.first().alias('topics_0'))
    .with_columns(
        pl.format('0x{}', pl.col('topics').list.get(1).str.slice(-40)
                  ).str.to_lowercase().alias('account'))
    .group_by("topics_0")
    .agg([pl.len().alias("count")])
    .with_columns((pl.col("count").truediv(pl.sum("count"))).mul(100).alias("percent_count"))
    .sort("count", descending=True)
    .with_columns(pl.col("topics_0").map_elements(lambda x: events_dict.get(x)['name'], pl.String).alias("event_name"))
).collect(new_streaming=True)

topics_0,count,percent_count,event_name
str,u32,f64,str
"""0x87aeeb9eda09a064caef63d00f62…",625143,5.173799911610330327e1,"""CanClaim"""
"""0x8629b200ebe43db58ad688b85131…",583137,4.826150431272066754e1,"""HasClaimed"""
"""0x8be0079c531659141344cd1fd0a4…",3,2.482855880147580860e-4,"""OwnershipTransferred"""
"""0xbea8251f76064f657f2a744bf08a…",2,1.655237253431720483e-4,"""SweepReceiverSet"""
"""0x7f221332ee403570bf4d61630b58…",1,8.276186267158602416e-5,"""Swept"""


In [46]:
file_path = os.path.realpath(os.path.join(
    path_data, "arbitrum_transfer.parquet"))
arbitrum_transfer_pl.write_parquet(file_path, compression="gzip")

file_path = os.path.realpath(os.path.join(
    path_data, "arbitrum_claim.parquet"))
arbitrum_has_claimed_pl.write_parquet(file_path, compression="gzip")

file_path = os.path.realpath(os.path.join(
    path_data, "arbitrum_can_claim.parquet"))
arbitrum_can_claim_pl.write_parquet(file_path, compression="gzip")

## Optimism


In [47]:
path_optimism = os.path.realpath(os.path.join(
    path_data, "optimism"))
transfer_event = "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef"
has_claimed_event = "0x4ec90e965519d92681267467f775ada5bd214aa92c0dc93d90a5e880ce9ed026"
op_decimals = Utils.load_contract_settings(
    chain="optimism")['optimism-token']['decimals']

In [48]:
column_order = ['block_number', 'block_hash', 'transaction_hash',
                'transaction_index', 'log_index', 'address', 'data', 'topics', 'block_timestamp']
schema = {
    'block_number': pl.Int64,
    'block_hash': pl.String,
    'transaction_hash': pl.String,
    'transaction_index': pl.Int64,
    'log_index': pl.Int64,
    'address': pl.String,
    'data': pl.String,
    'topics': pl.String,
    'block_timestamp': pl.String,
}

In [49]:
filepath = os.path.join(path_optimism, "csv", "transfer")
filenames = [filename for filename in os.listdir(
    filepath) if filename.startswith('op_') and filename.endswith('.csv.gz')]
dfs = list()
for filename in tqdm(filenames):
    file_dir = os.path.join(filepath, filename)
    df = pd.read_csv(file_dir, compression='gzip')
    dfs.append(df)
df = pd.concat(dfs)
df = pl.DataFrame(df[column_order], schema=schema)

file_dir = os.path.join(path_optimism, "optimism_transfer.parquet")
df.write_parquet(file_dir, compression='gzip')

  0%|          | 0/100 [00:00<?, ?it/s]

In [50]:
filepath = os.path.join(path_optimism, "csv", "logs")
filenames = [filename for filename in os.listdir(
    filepath) if filename.startswith('op_') and filename.endswith('.csv.gz')]
dfs = list()
for filename in tqdm(filenames):
    file_dir = os.path.join(filepath, filename)
    df = pd.read_csv(file_dir, compression='gzip')
    dfs.append(df)
df = pd.concat(dfs)
df = pl.DataFrame(df[column_order], schema=schema)

file_dir = os.path.join(path_optimism, "optimism_logs.parquet")
df.write_parquet(file_dir, compression='gzip')

  0%|          | 0/74 [00:00<?, ?it/s]

In [51]:
# Parse dataframe
file_path = os.path.realpath(os.path.join(
    path_optimism, "optimism_transfer.parquet"))
optimism_transfer_pl = (
    pl.scan_parquet(file_path)
    .with_columns(
        pl.col("topics").str.split(","))
    .with_columns(pl.col('topics').list.first().alias('topics_0'))
    .filter(pl.col('topics_0').eq(transfer_event))
    .with_columns(pl.format('0x{}', pl.col('topics').list.get(1).str.slice(-40)
                            ).str.to_lowercase().alias('from'),
                  pl.format('0x{}', pl.col('topics').list.get(2).str.slice(-40)
                            ).str.to_lowercase().alias('to'),
                  pl.col('data').str.replace('0x', '0x0').map_elements(
        Utils.parse_amount, pl.Float64).truediv(10**op_decimals).alias('amount'),
        pl.from_epoch(pl.col("block_timestamp"),
                      time_unit="s").alias('timestamp'),
    )
    .sort(by=['block_number', 'transaction_index', 'log_index'], descending=False)
    .select([pl.col("block_number").alias("blockNumber"),
             pl.col("transaction_hash").alias("transactionHash"),
             pl.col("block_hash").alias("blockHash"),
             pl.col('address'),
             pl.col("transaction_index").alias("transactionIndex"),
             pl.col("log_index").alias("logIndex"),
             pl.col("from"), pl.col("to"),
             pl.col("amount"),
             pl.col("timestamp"),
             ])

).collect(new_streaming=True)

optimism_transfer_pl.head()

blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,from,to,amount,timestamp
i64,str,str,str,i64,i64,str,str,f64,datetime[μs]
6491116,"""0x28a8a9378512c4dfa2add2f09a59…","""0x17c152d8fb7d0d328648011ab98b…","""0x4200000000000000000000000000…",0,0,"""0x0000000000000000000000000000…","""0xab1ef74d2c461e95f4b658ca1f94…",4.294967296000000000e9,2022-04-26 13:42:36
6493501,"""0x06704495a4fdba5e34653c457a73…","""0x89ce6d9c5ad45bc51b7c5cf75e23…","""0x4200000000000000000000000000…",0,0,"""0xab1ef74d2c461e95f4b658ca1f94…","""0x2a82ae142b2e62cb7d10b55e323a…",1.460288880000000238e9,2022-04-26 14:24:08
6493505,"""0x6e5d78cb27a0d74e414b1c084712…","""0x7c37e9b91ea95eea34e2f0ee897c…","""0x4200000000000000000000000000…",0,0,"""0xab1ef74d2c461e95f4b658ca1f94…","""0x2501c477d0a35545a387aa4a3eee…",1.288490189000000238e9,2022-04-26 14:24:08
6493508,"""0x63c2e58756ffebbee7e2d47dfdf2…","""0x955bd5180f3520079f0a34754134…","""0x4200000000000000000000000000…",0,0,"""0xab1ef74d2c461e95f4b658ca1f94…","""0x1ea3e20a3d0f9cb2cc6e4a5ae8e8…",1.546188227000000238e9,2022-04-26 14:24:23
6509766,"""0x83aea77fd50bee86b7f00e4a395a…","""0x58389db904829bf9ae0f63fa37c2…","""0x4200000000000000000000000000…",0,1,"""0x1ea3e20a3d0f9cb2cc6e4a5ae8e8…","""0xf348f595366fb381e705c28c3347…",1.817958000000000000e6,2022-04-26 18:12:55


In [52]:
optimism_claiming_address = "0xfedfaf1a10335448b7fa0268f56d2b44dbd357de"
op_gnosis_safe = "0x2a82ae142b2e62cb7d10b55e323acb1cab663a26"
optimism_claim_pl = optimism_transfer_pl.filter(
    pl.col("from").eq(optimism_claiming_address) & pl.col("to").ne(op_gnosis_safe))
print('There are {} Claim events'.format(len(optimism_claim_pl)))
optimism_claim_pl.head()

There are 160603 Claim events


blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,from,to,amount,timestamp
i64,str,str,str,i64,i64,str,str,f64,datetime[μs]
9401026,"""0x8214c65e61dde4d78b2d7ab75446…","""0x6e72940ebf84ca8def45f9df4b51…","""0x4200000000000000000000000000…",0,0,"""0xfedfaf1a10335448b7fa0268f56d…","""0xa902a27a7631d502e3ec17fc5d4c…",1.190267702142779171e3,2022-05-31 14:39:05
9401072,"""0x4e97db80526f12e9574d44e9de8f…","""0x5619802f3a5090c8807cc6efe377…","""0x4200000000000000000000000000…",0,0,"""0xfedfaf1a10335448b7fa0268f56d…","""0x69dc97ad2dc7b179bff12a6eadab…",2.469358092149939239e3,2022-05-31 14:39:50
9401125,"""0x8d9c6fc69b82df0e54bb472a2102…","""0x74c80acdabbf65b4d7134efef22d…","""0x4200000000000000000000000000…",0,0,"""0xfedfaf1a10335448b7fa0268f56d…","""0x68108902de3a5031197a6eb3b74b…",2.469358092149939239e3,2022-05-31 14:40:22
9401330,"""0x65073aa5512eca6027656aff99f6…","""0xc28b199d12dd87138cb05441292b…","""0x4200000000000000000000000000…",0,0,"""0xfedfaf1a10335448b7fa0268f56d…","""0xe56e30a76f08e67a72dd9f360e42…",2.469358092149939239e3,2022-05-31 14:42:27
9401349,"""0xdb52a8c4285ee25f09e77ea78bd3…","""0xc2f2444b0349a4b5152c4a79badb…","""0x4200000000000000000000000000…",0,0,"""0xfedfaf1a10335448b7fa0268f56d…","""0x992249d72114aac8ecae64245278…",9.620726743000000616e-8,2022-05-31 14:42:42


In [53]:
file_path = os.path.realpath(os.path.join(
    path_data, "optimism_transfer.parquet"))
optimism_transfer_pl.write_parquet(file_path, compression="gzip")

file_path = os.path.realpath(os.path.join(
    path_data, "optimism_claim.parquet"))
optimism_claim_pl.write_parquet(file_path, compression="gzip")